<a href="https://colab.research.google.com/github/ganeshred/NLPAuthorshipAttributionNLM/blob/main/bi_LSTM_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torchtext==0.10.1

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/gdrive/MyDrive/balanced_p1.csv')

In [ ]:
len(df)

4264

In [ ]:
df.head()

,Unnamed: 0,T1,T2,class
0,0,"Cities deserted, families separated and social...",Latest Headlines on CNN Business the great shu...,1.0
1,1,Working from home is a lifesaver -- and a big ...,China wants to take a victory lap over its han...,1.0
2,2,Meet the 2019 CNN Hero of the Year (cnn) it's ...,Coronavirus disinformation creates challenges ...,1.0
3,3,One of India's biggest banks has been rescued ...,China coronavirus: Eating wild animals made il...,1.0
4,4,Global Car Industry getty images tesla has rev...,China's economy could shrink for the first tim...,1.0


In [ ]:
df['text'] = df['T1']+df['T2']

In [ ]:
df.head()

,Unnamed: 0,T1,T2,class,text
0,0,"Cities deserted, families separated and social...",Latest Headlines on CNN Business the great shu...,1.0,"Cities deserted, families separated and social..."
1,1,Working from home is a lifesaver -- and a big ...,China wants to take a victory lap over its han...,1.0,Working from home is a lifesaver -- and a big ...
2,2,Meet the 2019 CNN Hero of the Year (cnn) it's ...,Coronavirus disinformation creates challenges ...,1.0,Meet the 2019 CNN Hero of the Year (cnn) it's ...
3,3,One of India's biggest banks has been rescued ...,China coronavirus: Eating wild animals made il...,1.0,One of India's biggest banks has been rescued ...
4,4,Global Car Industry getty images tesla has rev...,China's economy could shrink for the first tim...,1.0,Global Car Industry getty images tesla has rev...


In [ ]:


df = df[['text','class']]

In [ ]:
df[['text']] = df[['text']]. astype(str)

<ipython-input-11-42dfdfbe4dfa>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['text']] = df[['text']]. astype(str)


In [ ]:
# text_field = data.Field(
#     sequential=True,
#     tokenize='basic_english', 
#     fix_length=500,
#     lower=True
# )
TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(sequential=False, use_vocab=False, dtype = torch.float)

# label_field = data.Field(sequential=False, use_vocab=False)

# sadly have to apply preprocess manually
preprocessed_text = df['text'].apply(
    lambda x: TEXT.preprocess(x)
)

# load fastext simple embedding with 300d
TEXT.build_vocab(
    preprocessed_text, 
    max_size = 20000,
    vectors='glove.6B.50d',
    unk_init = torch.Tensor.normal_
)

# get the vocab instance
vocab = TEXT.vocab

/usr/local/lib/python3.9/dist-packages/torchtext/data/utils.py:123: UserWarning: Spacy model "en" could not be loaded, trying "en_core_web_sm" instead
  warnings.warn(f'Spacy model "{language}" could not be loaded, trying "{OLD_MODEL_SHORTCUTS[language]}" instead')


In [ ]:
ltoi = {l: i for i, l in enumerate(df['class'].unique())}

In [ ]:
df['label1'] = df['class'].apply(lambda y: ltoi[y])

class DataFrameDataset(data.Dataset):
    def __init__(self, df: pd.DataFrame, fields: list):
        super(DataFrameDataset, self).__init__(
            [
                data.Example.fromlist(list(r), fields) 
                for i, r in df.iterrows()
            ], 
            fields
        )

In [ ]:
df1=df[['text','label1']]

In [ ]:
examples=[]
for i,row in df1.iterrows():
  # print(example)
  examples.append(data.Example.fromlist(row, fields=[('text', TEXT), ('label', LABEL)]))

In [ ]:
# examples[2000].label
c=0
for i in examples:
  if type(i.label) != int:
    c+=1

In [ ]:
c

0

In [ ]:
dataset = data.Dataset(examples, fields=[('text', TEXT), ('label', LABEL)])

In [ ]:
train_dataset, test_dataset, valid_dataset = dataset.split(split_ratio=[0.6, 0.2, 0.2], stratified=True, strata_field='label')

In [ ]:
len(train_dataset)

2558

In [ ]:
ltoi

{1.0: 0, 0.0: 1}

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("We are working with ", device)
# device = 'cpu'
train_iter, test_iter, valid_iter = data.BucketIterator.splits(
    datasets=(train_dataset, test_dataset, valid_dataset), 
    batch_size=32,
    sort_within_batch = True,
    sort_key=lambda x: len(x.text),
    device=device
)

We are working with  cuda


In [ ]:
import torch.nn as nn

class LSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        super().__init__()
        #1. Initialize Embedding Layer
        self.embedding = nn.Embedding(num_embeddings = vocab_size, embedding_dim = embedding_dim, padding_idx = pad_idx)
        #2. Initialize LSTM layer
        self.lstm = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_dim, num_layers = n_layers, dropout = dropout, bidirectional = bidirectional)
        #3. Initialize a fully connected layer with Linear transformation
        self.fc = nn.Linear(hidden_dim * n_layers, output_dim)
        #4. Initialize Dropout
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        #1. Apply embedding layer that matches each word to its vector and apply dropout. Dim [sent_len, batch_size, emb_dim]
        embedding = self.embedding(text)
        embedding_dropout = self.dropout(embedding)
        #2. Run the LSTM along the sentences of length sent_len. #output = [sent len, batch size, hid dim * num directions]; #hidden = [num layers * num directions, batch size, hid dim]
        output, (hidden,cell) = self.lstm(embedding_dropout)
        #3. Concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        concat = torch.cat((hidden[-2, :, :], hidden[-1, :, :]), dim=1)
        hidden = self.dropout(concat)     
        return self.fc(hidden)

In [ ]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 50
HIDDEN_DIM = 126
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = LSTM(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT,
            PAD_IDX)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,562,817 trainable parameters


In [ ]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([20002, 50])


In [ ]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  0.7372,  0.3910,  0.5158],
        [ 0.1825, -0.0737,  0.3147,  ...,  1.3226, -2.6796, -0.1297],
        [ 0.0134,  0.2368, -0.1690,  ..., -0.5666,  0.0447,  0.3039],
        ...,
        [ 0.2278, -0.2761,  0.2514,  ..., -0.3299,  0.4107,  0.3511],
        [ 0.7214, -0.6658, -0.7967,  ..., -0.1520, -0.0712,  0.2026],
        [ 0.0873,  0.4776, -1.3107,  ..., -0.0543, -0.3408, -0.3862]])

In [ ]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0134,  0.2368, -0.1690,  ..., -0.5666,  0.0447,  0.3039],
        ...,
        [ 0.2278, -0.2761,  0.2514,  ..., -0.3299,  0.4107,  0.3511],
        [ 0.7214, -0.6658, -0.7967,  ..., -0.1520, -0.0712,  0.2026],
        [ 0.0873,  0.4776, -1.3107,  ..., -0.0543, -0.3408, -0.3862]])


In [ ]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [ ]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCEWithLogitsLoss()
# device='cpu'
model = model.to(device)
criterion = criterion.to(device)

In [ ]:
# def binary_accuracy(preds, y):
#     """
#     Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
#     """
#     # get the index of the highest prediction for each instance
#     _, preds = torch.max(preds, dim=1)
#     correct = (preds == y).float() #convert into float for division 
#     acc = correct.sum() / len(correct)
#     return acc

def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    # predicted_labels = torch.argmax(rounded_preds, dim=1)

    # print(predicted_labels.shape)
    # print(y.shape)
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [ ]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        # print(batch.text)
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)

        target = batch.label
        # target = target.type(torch.LongTensor)

        # print(target)

        # print(predictions)
        
        loss = criterion(predictions.to(device), target.to(device))
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        for batch in iterator:
            
            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            target = batch.label
            # target = target.type(torch.LongTensor)
            
            loss = criterion(predictions.to(device), target.to(device))
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
            
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 10
path = F"/content/gdrive/My Drive/bilstm_model_task1.pt"
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iter, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), path)
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 26s
	Train Loss: 0.596 | Train Acc: 68.11%
	 Val. Loss: 0.757 |  Val. Acc: 47.69%
Epoch: 02 | Epoch Time: 0m 26s
	Train Loss: 0.576 | Train Acc: 69.82%
	 Val. Loss: 0.781 |  Val. Acc: 45.72%
Epoch: 03 | Epoch Time: 0m 25s
	Train Loss: 0.566 | Train Acc: 70.17%
	 Val. Loss: 0.807 |  Val. Acc: 47.38%
Epoch: 04 | Epoch Time: 0m 25s
	Train Loss: 0.538 | Train Acc: 72.75%
	 Val. Loss: 0.868 |  Val. Acc: 46.06%
Epoch: 05 | Epoch Time: 0m 25s
	Train Loss: 0.521 | Train Acc: 73.53%
	 Val. Loss: 0.890 |  Val. Acc: 46.18%
Epoch: 06 | Epoch Time: 0m 26s
	Train Loss: 0.499 | Train Acc: 75.92%
	 Val. Loss: 0.897 |  Val. Acc: 45.37%
Epoch: 07 | Epoch Time: 0m 26s
	Train Loss: 0.477 | Train Acc: 75.15%
	 Val. Loss: 0.939 |  Val. Acc: 45.83%
Epoch: 08 | Epoch Time: 0m 24s
	Train Loss: 0.462 | Train Acc: 76.93%
	 Val. Loss: 1.042 |  Val. Acc: 46.18%
Epoch: 09 | Epoch Time: 0m 26s
	Train Loss: 0.447 | Train Acc: 79.20%
	 Val. Loss: 0.956 |  Val. Acc: 43.63%
Epoch: 10 | Epoch T

In [ ]:
test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 1.043 | Test Acc: 43.40%
